In [1]:
import json
import os
import numpy as np
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import XMLParser
import xml.etree.ElementTree as ET

# Specify the directory containing the JSON files
directory = r'C:\Projects\supervisely-pascalvoc-function'

In [8]:
def getOutput():
        for filename in os.listdir(directory):
            # Check if the file is a JSON file
            if filename.endswith(".xml"):
                #print(filename)
                empty_dict = {"description": "","tags": []}
                tree = ET.parse(filename)
                root = tree.getroot()
                #print(root.tag)
                objects_array = []
                for child in root:
                    #print(child.tag, child.text)
                    if child.tag=='size':
                        size_dict = {}
                        #print(len(child))
                        for c in child:
                            #print(c.tag, c.text)
                            if c.tag=='height' or c.tag=='width':
                                #print(c.text)
                                size_dict.update({c.tag: c.text.strip()})
                        #print(size_dict)
                        empty_dict[child.tag]= size_dict
                        #print(empty_dict)
                    elif child.tag=='object':
                        object_dict= {}
                        #print(len(child))
                        for c in child:
                            #print(c.tag)
                            if c.tag=='name':
                                object_dict.update({'classTitle': c.text.strip()})
                            elif c.tag=='bndbox':
                                #print(len(c))
                                x1= None
                                x2= None
                                y1= None
                                y2= None
                                for cc in c:
                                    #print(cc.tag)
                                    #print(cc.tag)
                                    if cc.tag=='xmax':
                                        x2=cc.text.strip()
                                    elif cc.tag=='xmin':
                                        x1=cc.text.strip()
                                    elif cc.tag=='ymax':
                                        y2=cc.text.strip()
                                    elif cc.tag=='ymin':
                                        y1=cc.text.strip()
                                exterior = [[int(x1), int(y1)], [int(x2), int(y2)]]
                                #print(exterior)
                                interior = []
                                object_dict['points'] = {"exterior": exterior, "interior": interior}
                            object_dict['geometryType'] = 'rectangle'
                        #print(object_dict)
                        objects_array.append(object_dict)
                #print(len(objects_array))
                empty_dict['objects']= objects_array
                print(empty_dict)
                # Combine the filename with the base filename to create the full filename
                formatted = "pascal_to_supervisely_" + filename+".json"
                print(formatted)
                # Specify the directory name
                directory_name = "output-directory"
                
                # Create the directory if it doesn't exist
                if not os.path.exists(directory_name):
                    os.makedirs(directory_name)
                # Construct the full file path
                filepath = os.path.join(directory_name, formatted)
                # Save the JSON data to the file
                with open(filepath, "w") as f:
                    json.dump(empty_dict, f, indent=4)

            elif filename.endswith(".json"):
                root_node = Element('annotation')
                extracted_filename = filename.split(".json")[0]
                #print("filename: ", extracted_filename)
                filepath = os.path.join(directory, filename)
                segmented= '0'
                folder= '0'
                #print("folder: ", folder)
                annotation = 'PASCAL VOC2010'
                database = 'The VOC2010 Database'
                image= 'flickr'
                # Open the JSON file
                with open(filepath, "r") as f:
                    # Load the JSON data
                    data = json.load(f)
                    width = data['size']['width']
                    height= data['size']['height']
                    depth= 0
                    #dictionary = {"filename": extracted_filename, "folder": folder, "size": {"depth": depth, "height": height, "width": width}, "source": {"annotation": annotation, "database": database, "image": image}}
                    #print("Depth: ", depth, "Height: ", height, "Width: ", width)
                    #print(dictionary)
                    #print(width, height)
                    objects = data['objects']
                    objects_dict = []
                    for object in objects:
                        points= np.array(object['points']['exterior'])
                        # Extract x1, y1, x2, y2
                        x1 = points[0, 0]
                        y1 = points[0, 1]
                        x2 = points[1, 0]
                        y2 = points[1, 1]
                        name= object['classTitle']
                        #print(classTitle)
                        #print("bbox: ", x1, y1, x2, y2)
                        pose="Unspecified"
                        truncated= 0
                        difficult= 0
                        occluded= 0
                        #print("name: ", name, "Pose: ", pose, "Truncated: ",truncated, "Occluded: ", occluded)
                        #print("Segmented: ", segmented)
                        #print("Annotation: ", annotation, "Database: ", database, "Image: ", image)
                        objects_dict.append({"name": name, "bndbox": {"xmax": str(x2), "xmin": str(x1), "ymax": str(y2), "ymin": str(y1)}, "difficult": str(difficult), "occluded": str(occluded), "pose": pose, "truncated": str(truncated)})
                        #dictionary['object']={"name": name, "bndbox": {"xmax": x2, "xmin": x1, "ymax": y2, "ymin": y1}, "difficult": difficult, "occluded": occluded, "pose": pose, "truncated": truncated}
                    #print(objects_dict)
                dictionary = {"filename": extracted_filename, "folder": folder, "segmented": segmented, "size": {"depth": str(depth), "height": str(height), "width": str(width)}, "source": {"annotation": annotation, "database": database, "image": image}, "objects": objects_dict}
                #print(dictionary) 
                
                for key, value in dictionary.items():
                    #print(key, value)
                    if key== 'filename':
                        child_node = Element(key)
                        child_node.text = value
                        root_node.append(child_node)
                        #print(builder.set(f'/annotation/filename[name=key]', value))
                    elif key== 'folder':
                        child_node = Element(key)
                        child_node.text = value
                        root_node.append(child_node)
                    elif key== 'size':
                        x= value
                        #print(x)
                        size_node = Element('size')
                        
                        for key, value in x.items():
                            if key=='depth':
                                child_node = Element(key)
                                child_node.text = value
                                size_node.append(child_node)
                            elif key=='height':
                                child_node = Element(key)
                                child_node.text = value
                                size_node.append(child_node)
                            elif key=='width':
                                child_node = Element(key)
                                child_node.text = value
                                size_node.append(child_node)
                        #ET.dump(size_node)
                        root_node.append(size_node)
        
                    elif key== 'source':
                        x= value
                        #print(x)
                        source_node = Element('source')
                        
                        for key, value in x.items():
                            if key=='annotation':
                                child_node = Element(key)
                                child_node.text = value
                                source_node.append(child_node)
                            elif key=='database':
                                child_node = Element(key)
                                child_node.text = value
                                source_node.append(child_node)
                            elif key=='image':
                                child_node = Element(key)
                                child_node.text = value
                                source_node.append(child_node)
                        #ET.dump(source_node)
                        root_node.append(source_node)
                    elif key== 'segmented':
                        child_node = Element(key)
                        child_node.text = value
                        root_node.append(child_node)
                    elif key=='objects':
                        #print(value)
                        objects = value
                        for object in objects:
                            #print(object)
                            object_node= Element('object')
                            for key, value in object.items():
                                #print(key, value)
                                if key== 'name':
                                    child_node = Element(key)
                                    child_node.text = value
                                    object_node.append(child_node)
                                elif key== 'bndbox':
                                    #print(value)
                                    bndbox_node = Element('bndbox')
                                    bndbox= value
                                    for key, value in bndbox.items():
                                        if key== 'xmax':
                                            child_node = Element(key)
                                            child_node.text = value
                                            bndbox_node.append(child_node)
                                        elif key== 'xmin':
                                            child_node = Element(key)
                                            child_node.text = value
                                            bndbox_node.append(child_node)
                                        elif key== 'ymax':
                                            child_node = Element(key)
                                            child_node.text = value
                                            bndbox_node.append(child_node)
                                        elif key== 'ymin':
                                            child_node = Element(key)
                                            child_node.text = value
                                            bndbox_node.append(child_node)
                                    object_node.append(bndbox_node)
                                elif key== 'difficult':
                                    child_node = Element(key)
                                    child_node.text = value
                                    object_node.append(child_node)
                                elif key== 'occluded':
                                    child_node = Element(key)
                                    child_node.text = value
                                    object_node.append(child_node)
                                elif key== 'pose':
                                    child_node = Element(key)
                                    child_node.text = value
                                    object_node.append(child_node)
                                elif key== 'truncated':
                                    child_node = Element(key)
                                    child_node.text = value
                                    object_node.append(child_node)
                            #ET.dump(object_node)
                            root_node.append(object_node)
                            #print(root_node.text)
                    
                
                #xml_data = ET.dump(root_node)
                xml_str = ET.tostring(root_node).decode()
                print(xml_str)
                formatted = "supervisely_to_pascal_" + filename+".xml"
                print(formatted)
                # Specify the directory name
                directory_name = "output-directory"
                
                # Create the directory if it doesn't exist
                if not os.path.exists(directory_name):
                    os.makedirs(directory_name)
                # Construct the full file path
                filepath = os.path.join(directory_name, formatted)
                # Write the output to a file (optional)
                with open(filepath, mode="w", encoding="utf-8") as out_file:
                  out_file.write(xml_str)

            else:
                print("this file/folder can not be formatted into pascalvoc or supervisely format")
                

In [9]:
getOutput()

this file can not be formatted into pascalvoc or supervisely format
this file can not be formatted into pascalvoc or supervisely format
{'description': '', 'tags': [], 'size': {'height': '750', 'width': '1125'}, 'objects': [{'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[227, 8], [295, 84]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[473, 175], [545, 219]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[825, 608], [882, 659]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[1005, 620], [1044, 663]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[273, 604], [339, 662]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'exterior': [[251, 89], [319, 150]], 'interior': []}}, {'classTitle': 'bird', 'geometryType': 'rectangle', 'points': {'